# Clasificación de dígitos de la base de datos MNIST con Deep Learning 

### https://www.tensorflow.org/get_started/mnist/pros

Cargamos los datos en la clase mnist, que contiene los conjuntos de entrenamiento, test y validación en numpy arrays, y una función para iterar en los batches de datos.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


Iniciamos una sesión interactiva de TenorFlow, que es más flexible que una sesión normal a la hora de definir el gráfico mietnras se lanzan algunos procesos del mismo.

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

## Modelo de regresión Softmax

Definimos inputs y targets:

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784]) 
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# None indica "cualquier tamaño", la primera dimensión corresponde con el 
# batch size

Las dimensiones son un argumento opcional, pero ayudan a controlar errores dimensionales en el modelo.
Ahora definimos pesos y bias, y los inicializamos a cero:

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

Definimos el modelo, utilizando la funcion matmul() (multiplicación de matrices) y definimos la función de coste (loss function), utilizaremos cross entropy:

In [6]:
y = tf.matmul(x, W) + b
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

Entrenamos el modelo con el método gradient-descent, y step length de 0.5. train_step aplica las actualizaciones de gradient-descent a los parámetros, por lo que el modelo se entrena aplicando esta función iterativamente:

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Evaluamos el modelo. La función cast pasa de vector booleano [True, False, True, True] a floats [1,0,1,1], y reduce_mean lo promedia (0.75).

In [8]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9162


# Multilayer Convolutional Network

Vamos a definir una red profunda convolucional con la arquitectura:
    conv1-pool1-conv2-pool2-fc1-dropout-fc2
Para inicializar los pesos y bias de las diferentes capas creamos dos funciones. Es interesante que los valores iniciales tengan un cierto ruido aletaorio para evitar gradientes nulos y, puesto que utilizaremos neuronas con ReLu, conviene que los valores de bias sean ligeramente positivos: 

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Definimos las capas de convolución y pooling: convolución con pad=0 y stride=1 para mantener las dimensiones de entrada; y pooling max_pooling en bloques de 2x2.

In [10]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

Primera capa convolucional: 32 características cada patch de 5x5: [5, 5, 1, 32], los dos primeros valores son las dimensiones del patch, el siguiente el número de canales de entrada, y el cuarto el número de canales de salida. Hay que aplicar un reshape a la imagen para convertirla en un tensor de 4d, y aplicar la convolución de la imagen con el tensor de pesos. 
Finalmente se aplica la función ReLu y el maxPool

In [11]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1]) # [new dim, dimx, dimy, colors]

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Construimos la segunda capa convolucional

In [12]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Construimos la fully connected layer.
Con la imagen reducida a 7x7, construimos una capa de 1024 neuronas totalmente conectadas a las entradas. La imagen se ha reducido de 28x28 a 7x7 por aplicar las dos capas de pooling ¿?

In [13]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Dropout. Para minimizar el overfitting aplicamos dropout antes de la capa de salida. Creamos la variable keep_prob que indicará la probabilidad de que una neurona sea eliminada en el dropout, lo que permite aplicar dropout en el entrenamiento, pero no aplicarlo en el testing.

In [14]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Capa de salida (readout layer)

In [15]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Entrenamiento y evaluación del modelo

Vamos a utilizar, en lugar del gradient descent, el optimizador ADAM, aplicamos el dropout rate con el parámetro keep_proba, e indicamos loggin cada 100 iteraciones.

In [16]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob:0.5
            })
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0  }))


step 0, training accuracy 0.18
step 100, training accuracy 0.78
step 200, training accuracy 0.86
step 300, training accuracy 0.82
step 400, training accuracy 0.9
step 500, training accuracy 0.9
step 600, training accuracy 0.88
step 700, training accuracy 0.92
step 800, training accuracy 0.96
step 900, training accuracy 0.92
step 1000, training accuracy 0.82
step 1100, training accuracy 0.94
step 1200, training accuracy 0.94
step 1300, training accuracy 0.94
step 1400, training accuracy 0.96
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.94
step 1800, training accuracy 0.98
step 1900, training accuracy 0.92
test accuracy 0.9723
